In [ ]:
#pip install holidays

In [ ]:
# Basic imports
import numpy as np
import pandas as pd
import datetime # manipulating date formats
import itertools
import time
import holidays
import os

from math import sqrt
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE, r2_score, mean_absolute_percentage_error as MAPE
from sklearn import datasets, linear_model

# Stats packages
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.tools import diff
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
nyc_v = pd.read_csv("/Users/sameshbajaj/Desktop/Time Series Analysis/Assignments/Project/nyc_violent_final copy.csv", parse_dates=['date'])
nyc_temp = pd.read_csv("/Users/sameshbajaj/Desktop/Time Series Analysis/Assignments/Project/ISYE6203_NYC_avg_temp_2009-2020.csv", parse_dates=['date'])
holiday = pd.read_csv("/Users/sameshbajaj/Desktop/Time Series Analysis/Assignments/Project/Holidays_2009-2020.csv", parse_dates=['date'])

In [ ]:
nyc_v = nyc_v.set_index("date")
nyc_temp = nyc_temp.set_index("date")
holiday = holiday.set_index("date")

In [ ]:
nyc_v.plot()

In [ ]:
nyc_temp

In [ ]:
nyc_temp.plot()

In [ ]:
holiday

In [ ]:
def test_stationarity(timeseries):
    # Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries,maxlag=7*4, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (round(dfoutput,3))

In [ ]:
test_stationarity(nyc_v)

In [ ]:
acf=plot_acf(nyc_v)
pacf=plot_pacf(nyc_v)

In [ ]:
train_end = '2020-12-24'
#test_start = '2020-12-18'
test_end = '2020-12-31'
demo_start = '2020-10-31'
demo = nyc_v[demo_start:test_end]
nyc_v_train, nyc_v_test = nyc_v[:train_end], nyc_v[train_end:]

nyc_v_train.plot(figsize=(12,6), style='o', grid=True)

In [ ]:
begin = '2009-01-01'
abv = '2020-12-30'

exo = holiday.copy()
exo_train,exo_test = exo[:train_end], exo[train_end:abv]
exo_test

In [ ]:
exo_train.shape

In [ ]:
exo_test.shape

In [ ]:
nyc_v_train.shape

In [ ]:
nyc_v_test.shape

In [ ]:
def sarimax(ts,exo,all_param):
    results = []
    for param in all_param:
        try:
            mod = SARIMAX(ts,
                          exog = exo,
                          order=param[0],
                          seasonal_order=param[1])
            res = mod.fit()
            results.append((res,res.aic,param))
            print('Tried out SARIMAX{}x{} - AIC:{}'.format(param[0], param[1], round(res.aic,2)))
        except Exception as e:
            print(e)
            continue
            
    return results

In [ ]:
# set parameter range
p,d,q = range(0,2),[1],range(0,2)
P,D,Q,s = range(0,2),[1],range(0,2),[7]
# list of all parameter combos
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(P, D, Q, s))
all_param = list(itertools.product(pdq,seasonal_pdq))

In [ ]:
all_res = sarimax(nyc_v_train,exo_train,all_param)

In [ ]:
all_res.sort(key=lambda x: x[1])
all_res[:5]

In [ ]:
# set parameter range
p,d,q = range(0,3),[0],range(0,3)
P,D,Q,s = range(0,3),[0],range(0,3),[7]
# list of all parameter combos
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(P, D, Q, s))
all_param = list(itertools.product(pdq,seasonal_pdq))

In [ ]:
res = all_res[0][0]
res.plot_diagnostics(figsize=(15, 12))

plt.show()
print("Ljung-box p-values:\n" + str(res.test_serial_correlation(method='ljungbox')[0][1]))
res.summary()

In [ ]:
def pm(y_true, y_pred):
    return sum((y_true-y_pred)**2)/sum((y_true-np.mean(y_true))**2)

In [ ]:
pred_test = res.get_prediction(start=train_end,end=test_end,exog=exo_test)
err = ('\nPM: %.2f'% pm(nyc_v_test['crime count'], pred_test.predicted_mean.to_frame()['predicted_mean']) + \
      '\nMean absolute percentage error: %.2f'% MAPE(nyc_v_test, pred_test.predicted_mean) + \
      '\nRoot mean squared error: %.2f'% sqrt(MSE(nyc_v_test, pred_test.predicted_mean)))

pred = res.get_prediction(start=begin,end=test_end,exog=exo_test)
pred_ci = pred.conf_int()

fig, ax = plt.subplots(figsize=(12,7))
ax.set(title='NYC Violent Crime', ylabel='Number of Incidents')

nyc_v.plot(ax=ax, style = 'o')
pred.predicted_mean.plot(ax=ax, style='o')
ci = pred_ci.loc[demo_start:]
ax.fill_between(ci.index, ci.iloc[:,0], ci.iloc[:,1], color='r', alpha=0.1)

plt.figtext(0.12, -0.06, err, ha="left",fontsize=15,va='center')
legend = ax.legend(["Train Set Observed","Test Set Observed", "Forecast"])



In [ ]:
pred_test = res.get_prediction(start=train_end,end=test_end,exog=exo_test)
# The root mean squared error
err = ('\nPM: %.2f'% pm(nyc_v_test['crime count'], pred_test.predicted_mean.to_frame()['predicted_mean']) + \
      '\nMean absolute percentage error: %.2f'% MAPE(nyc_v_test, pred_test.predicted_mean) + \
      '\nRoot mean squared error: %.2f'% sqrt(MSE(nyc_v_test, pred_test.predicted_mean)))

pred = res.get_prediction(start=demo_start,end=test_end,exog=exo_test)
pred_ci = pred.conf_int()

fig, ax = plt.subplots(figsize=(12,7))
ax.set(title='NYC Violent Crime', ylabel='Number of Incidents')

nyc_v_train[demo_start:].plot(ax=ax)
nyc_v_test.plot(ax=ax)
pred.predicted_mean.plot(ax=ax)
ci = pred_ci.loc[demo_start:]
ax.fill_between(ci.index, ci.iloc[:,0], ci.iloc[:,1], color='r', alpha=0.1)

plt.figtext(0.12, -0.06, err, ha="left",fontsize=15,va='center')
legend = ax.legend(["Train Set Observed","Test Set Observed", "Forecast"])
ax.grid(True)